In [8]:
import numpy as np
from scipy.special import erfc
from scipy.stats import chi2
from numpy.fft import fft

def load_binary_file(filepath):
    with open(filepath, 'r') as f:
        data = f.read().strip()
    return np.array([int(b) for b in data if b in '01'])

def frequency_test(bits):
    n = len(bits)
    s = np.sum(2 * bits - 1)
    p = erfc(abs(s) / np.sqrt(2 * n))
    return p

def block_frequency_test(bits, block_size=128):
    n = len(bits)
    if n < block_size * 20:
        return None
    num_blocks = n // block_size
    blocks = bits[:num_blocks * block_size].reshape((num_blocks, block_size))
    proportions = np.mean(blocks, axis=1)
    chi_sq = 4 * block_size * np.sum((proportions - 0.5) ** 2)
    p = chi2.sf(chi_sq, df=num_blocks)
    return p

def fft_test(bits):
    n = len(bits)
    if n < 1000:
        return None
    x = 2 * bits - 1
    s = np.abs(fft(x))
    s = s[:n // 2]
    T = np.sqrt(np.log(1 / 0.05) * n)
    N0 = 0.95 * n / 2
    N1 = np.sum(s < T)
    d = (N1 - N0) / np.sqrt(n * 0.95 * 0.05 / 4)
    p = erfc(abs(d) / np.sqrt(2))
    return p

def non_overlapping_template_test(bits, template='000000001'):
    m = len(template)
    n = len(bits)
    block_size = 1032  # NIST default for this template
    if n < block_size * 8:
        return None
    num_blocks = n // block_size
    blocks = bits[:num_blocks * block_size].reshape((num_blocks, block_size))
    pattern = np.array([int(b) for b in template])

    counts = []
    for block in blocks:
        count = 0
        i = 0
        while i <= block_size - m:
            if np.array_equal(block[i:i + m], pattern):
                count += 1
                i += m
            else:
                i += 1
        counts.append(count)

    mean = (block_size - m + 1) / (2 ** m)
    var = block_size * ((1 / (2 ** m)) - ((2 * m - 1) / (2 ** (2 * m))))
    chi_sq = sum((x - mean) ** 2 for x in counts) / var
    p = chi2.sf(chi_sq, df=num_blocks)
    return p

def run_nist_tests(bits, label):
    print(f"\n--- NIST-style Tests for {label} ---")
    tests = {
        "Frequency Test": frequency_test(bits),
        "Block Frequency": block_frequency_test(bits),
        "FFT (Spectral) Test": fft_test(bits),
        "Non-overlapping Template": non_overlapping_template_test(bits)
    }

    for test, p in tests.items():
        if p is None:
            print(f"{test:<30}: Not enough bits ❌")
        else:
            status = "PASS ✅" if p > 0.01 else "FAIL ❌"
            print(f"{test:<30}: p = {p:.5f} -> {status}")

# Main
if __name__ == "__main__":
    prng_bits = load_binary_file("3.txt")
    qrng_bits = load_binary_file("quantum-random.txt")

    run_nist_tests(prng_bits, "PRNG")
    run_nist_tests(qrng_bits, "QRNG")



--- NIST-style Tests for PRNG ---
Frequency Test                : p = 0.23014 -> PASS ✅
Block Frequency               : Not enough bits ❌
FFT (Spectral) Test           : Not enough bits ❌
Non-overlapping Template      : Not enough bits ❌

--- NIST-style Tests for QRNG ---
Frequency Test                : p = 0.97277 -> PASS ✅
Block Frequency               : p = 0.85215 -> PASS ✅
FFT (Spectral) Test           : p = 0.37207 -> PASS ✅
Non-overlapping Template      : p = 0.50193 -> PASS ✅
